In [4]:
!pip install duckdb --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
!pip install python-gdcm
!pip install pylibjpeg
!pip install pylibjpeg-libjpeg==2.2.0
!pip install pylibjpeg-openjpeg==2.3.0
!pip install matplotlib==3.10.3
!pip install scikit-learn==1.7.0
!pip install polars --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install pydicom

Looking in links: /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 110.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 31.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.7 MB/s eta 0:00:00:00:010:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 

## This is the working combination

In [5]:
!pip install tensorflow-io==0.37.1
!pip install tensorflow==2.16.1
!pip install tensorflow-tpu==2.16.1 --find-links=https://storage.googleapis.com/libtpu-tf-releases/index.html


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 883.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 70.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 4.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 87.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Succe

In [6]:
from pydicom import dcmread
from pydicom.dataset import FileDataset, FileMetaDataset
from pydicom.uid import generate_uid, ImplicitVRLittleEndian

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
import pickle
import gc
import ctypes
from pathlib import Path
import logging
import json
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor
import datetime
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras

E0000 00:00:1758954079.372906      10 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D0927 06:21:19.379485891      10 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0927 06:21:19.379496051      10 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0927 06:21:19.379498611      10 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0927 06:21:19.379500241      10 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0927 06:21:19.379501671      10 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

In [7]:
print(tf.__version__)
print(tfio.__version__)

2.16.1
0.37.1


In [8]:
# detect TPUs
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1758954100.765310      10 service.cc:145] XLA service 0x5ce43f9c0e40 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758954100.765347      10 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1758954100.765350      10 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1758954100.765352      10 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1758954100.765354      10 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1758954100.765355      10 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1758954100.765357      10 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1758954100.765358      10 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1758954100.765359      10 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [9]:
pl.Config(fmt_str_lengths=1000)
pl.Config.set_tbl_rows(1000)

polars.config.Config

In [10]:
allowed_tags = ['BitsAllocated', 'BitsStored', 'Rows', 'Columns', 'FrameOfReferenceUID', 'HighBit', 'ImageOrientationPatient'
                , 'ImagePositionPatient', 'InstanceNumber', 'Modality', 'PhotometricInterpretation'
                , 'PixelRepresentation', 'PixelSpacing', 'PlanarConfiguration', 'RescaleIntercept', 'RescaleSlope'
                , 'RescaleType', 'SamplesPerPixel', 'SliceThickness', 'SpacingBetweenSlices']

In [11]:
df_all_data = pl.read_parquet('/kaggle/input/rsna-aneurysm-train-metadata-suman/full_training_data.parquet')
print("Shape of training metadata", df_all_data.shape)
df_all_data.columns

Shape of training metadata (1001346, 24)


['file_name',
 'image_shape',
 'coordinates_x',
 'coordinates_y',
 'coordinates_f',
 'aneurysm_present_in_series',
 'aneurysm_present_in_image',
 'seriesinstanceuid',
 'patientage',
 'patientsex',
 'modality',
 'left_infraclinoid_internal_carotid_artery',
 'right_infraclinoid_internal_carotid_artery',
 'left_supraclinoid_internal_carotid_artery',
 'right_supraclinoid_internal_carotid_artery',
 'left_middle_cerebral_artery',
 'right_middle_cerebral_artery',
 'anterior_communicating_artery',
 'left_anterior_cerebral_artery',
 'right_anterior_cerebral_artery',
 'left_posterior_communicating_artery',
 'right_posterior_communicating_artery',
 'basilar_tip',
 'other_posterior_circulation']

In [12]:
#with tf.device('/device:TPU:0'):
root_folder = "/kaggle/input/rsna-intracranial-aneurysm-detection/series"

def create_full_image_path(row_data):
    return row_data['root_folder'] + '/' + row_data['seriesinstanceuid'] + '/' + row_data['file_name']

df_all_data = df_all_data.with_columns(pl.lit(root_folder).alias("root_folder"))

df_all_data = df_all_data.with_columns([
    pl.struct(pl.col("root_folder"), pl.col("seriesinstanceuid"), pl.col("file_name"))
    .map_elements(create_full_image_path, return_dtype=pl.String)
    .alias("full_image_path")
])

In [13]:
def extract_single_frame(multiframe_path, slice_number, output_path=None):
    """
    Extract a single frame from a multi-frame DICOM
    
    Args:
        multiframe_path: Path to multi-frame DICOM file
        slice_number: The slice number to extract (0-based index)
        output_path: Path to save the single-frame DICOM. If None, returns the dataset
    """
    try:
        # Read the multi-frame DICOM with force=True to handle potentially corrupted files
        multi_ds = dcmread(multiframe_path, force=True)
        
        # Verify it's a multi-frame image
        if not hasattr(multi_ds, 'NumberOfFrames'):
            raise ValueError("Input DICOM is not a multi-frame image")
        
        # Check if slice number is valid
        if slice_number >= multi_ds.NumberOfFrames:
            raise ValueError(f"Slice number {slice_number} is out of range. "
                           f"Image has {multi_ds.NumberOfFrames} frames")
        
        # Create new dataset for single frame
        single_ds = FileDataset(output_path or "temp.dcm", {}, 
                              file_meta=FileMetaDataset(), 
                              preamble=b"\0" * 128)
        
        # Copy attributes from multi-frame dataset
        attrs_to_copy = allowed_tags
        
        for attr in attrs_to_copy:
            if hasattr(multi_ds, attr):
                setattr(single_ds, attr, getattr(multi_ds, attr))
        
        # Generate new UIDs
        single_ds.SOPInstanceUID = generate_uid()
        single_ds.file_meta.MediaStorageSOPInstanceUID = single_ds.SOPInstanceUID
        
        # Set transfer syntax to uncompressed little endian
        single_ds.file_meta.TransferSyntaxUID = ImplicitVRLittleEndian
        single_ds.file_meta.MediaStorageSOPClassUID = multi_ds.file_meta.MediaStorageSOPClassUID
        if hasattr(multi_ds.file_meta, 'ImplementationClassUID'):
            single_ds.file_meta.ImplementationClassUID = multi_ds.file_meta.ImplementationClassUID
        
        # Set instance-specific attributes
        single_ds.InstanceNumber = slice_number + 1
        
        try:
            # Try to get pixel array directly
            pixel_array = multi_ds.pixel_array[slice_number]
        except Exception as e:
            #print(f"Warning: Could not directly access pixel_array: {e}")
            # Alternative approach: decompress and get pixels
            if hasattr(multi_ds, 'decompress'):
                multi_ds.decompress()
            pixel_array = multi_ds.pixel_array[slice_number]
        
        # Set pixel data
        single_ds.PixelData = pixel_array.tobytes()
        
        # Update image-specific attributes
        single_ds.NumberOfFrames = 1
        
        # Try to copy position and orientation
        try:
            if hasattr(multi_ds, 'PerFrameFunctionalGroupsSequence'):
                frame_content = multi_ds.PerFrameFunctionalGroupsSequence[slice_number]
                
                if hasattr(frame_content, 'PlanePositionSequence'):
                    position = frame_content.PlanePositionSequence[0].ImagePositionPatient
                    single_ds.ImagePositionPatient = position
                
                if hasattr(frame_content, 'PlaneOrientationSequence'):
                    orientation = frame_content.PlaneOrientationSequence[0].ImageOrientationPatient
                    single_ds.ImageOrientationPatient = orientation
        except Exception as e:
            #print(f"Warning: Could not copy position/orientation: {e}")
            raise
        
        # Add creation timestamp
        dt = datetime.datetime.now()
        single_ds.ContentDate = dt.strftime('%Y%m%d')
        single_ds.ContentTime = dt.strftime('%H%M%S.%f')
        
        # Save or return the dataset
        if output_path:
            single_ds.save_as(output_path, write_like_original=False)
            return None
        return single_ds
    
    except Exception as e:
        #print(f"Error extracting frame: {e}")
        raise

# Alternative version using different approach for compressed files
def extract_single_frame_alternative(multiframe_path, slice_number, output_path=None):
    """
    Alternative version for handling problematic files
    """
    try:
        # Read with force and stop before pixels
        multi_ds = dcmread(multiframe_path, force=True, stop_before_pixels=True)
        
        # Read pixel data separately
        with open(multiframe_path, 'rb') as f:
            multi_ds.PixelData = f.read()
        
        # Decompress if needed
        if hasattr(multi_ds, 'decompress'):
            multi_ds.decompress()
        
        # Get pixel array
        pixel_array = multi_ds.pixel_array[slice_number]
        
        # Create new dataset
        single_ds = FileDataset(output_path or "temp.dcm", {}, 
                              file_meta=FileMetaDataset(), 
                              preamble=b"\0" * 128)
        
        # Copy attributes (same as before)
        attrs_to_copy = allowed_tags
        
        for attr in attrs_to_copy:
            if hasattr(multi_ds, attr):
                setattr(single_ds, attr, getattr(multi_ds, attr))
        
        # Generate new UIDs
        single_ds.SOPInstanceUID = generate_uid()
        single_ds.file_meta.MediaStorageSOPInstanceUID = single_ds.SOPInstanceUID
        
        # Set transfer syntax to uncompressed little endian
        single_ds.file_meta.TransferSyntaxUID = ImplicitVRLittleEndian
        single_ds.file_meta.MediaStorageSOPClassUID = multi_ds.file_meta.MediaStorageSOPClassUID
        if hasattr(multi_ds.file_meta, 'ImplementationClassUID'):
            single_ds.file_meta.ImplementationClassUID = multi_ds.file_meta.ImplementationClassUID
        
        # Set instance-specific attributes
        single_ds.InstanceNumber = slice_number + 1
        
        try:
            # Try to get pixel array directly
            pixel_array = multi_ds.pixel_array[slice_number]
        except Exception as e:
            #print(f"Warning: Could not directly access pixel_array: {e}")
            # Alternative approach: decompress and get pixels
            if hasattr(multi_ds, 'decompress'):
                multi_ds.decompress()
            pixel_array = multi_ds.pixel_array[slice_number]
        
        # Set pixel data
        single_ds.PixelData = pixel_array.tobytes()
        
        # Update image-specific attributes
        single_ds.NumberOfFrames = 1
        
        # Try to copy position and orientation
        try:
            if hasattr(multi_ds, 'PerFrameFunctionalGroupsSequence'):
                frame_content = multi_ds.PerFrameFunctionalGroupsSequence[slice_number]
                
                if hasattr(frame_content, 'PlanePositionSequence'):
                    position = frame_content.PlanePositionSequence[0].ImagePositionPatient
                    single_ds.ImagePositionPatient = position
                
                if hasattr(frame_content, 'PlaneOrientationSequence'):
                    orientation = frame_content.PlaneOrientationSequence[0].ImageOrientationPatient
                    single_ds.ImageOrientationPatient = orientation
        except Exception as e:
            #print(f"Warning: Could not copy position/orientation: {e}")
            raise
        
        # Add creation timestamp
        dt = datetime.datetime.now()
        single_ds.ContentDate = dt.strftime('%Y%m%d')
        single_ds.ContentTime = dt.strftime('%H%M%S.%f')
        
        # Save or return the dataset
        if output_path:
            single_ds.save_as(output_path, write_like_original=False)
            return None
        return single_ds
        
    except Exception as e:
        #print(f"Error in alternative extraction: {e}")
        raise

# Function to try both methods
def safe_extract_single_frame(multiframe_path, slice_number, output_path=None):
    """
    Try both extraction methods
    """
    try:
        return extract_single_frame(multiframe_path, slice_number, output_path)
    except Exception as e:
        #print(f"Primary method failed: {e}")
        #print("Trying alternative method...")
        try:
            return extract_single_frame_alternative(multiframe_path, slice_number, output_path)
        except Exception as e2:
            #print(f"Alternative method also failed: {e2}")
            raise

# Version with zoom functionality
def load_and_view_single_slice_with_zoom(dcm_path, x_coord, y_coord, f_coord=None, zoom_size=100):
    """
    Load and display a single DICOM slice with crosshair and zoomed inset
    
    Args:
        dcm_path: Path to the DICOM file
        x_coord: x coordinate for the crosshair
        y_coord: y coordinate for the crosshair
        zoom_size: Size of the zoom window in pixels
    """
    # Read DICOM file
    if f_coord:
        ds = safe_extract_single_frame(dcm_path, f_coord)
    else:
        ds = dcmread(dcm_path)
    img = ds.pixel_array
    
    # Create figure and axes
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
    
    # Main image with crosshair
    ax1.imshow(img, cmap='gray')
    ax1.axvline(x=x_coord, color='red', alpha=0.5)
    ax1.axhline(y=y_coord, color='red', alpha=0.5)
    ax1.plot(x_coord, y_coord, 'r+', markersize=10, markeredgewidth=2)
    
    # Zoomed region
    x_start = int(max(0, x_coord - zoom_size/2))
    x_end = int(min(img.shape[1], x_coord + zoom_size/2))
    y_start = int(max(0, y_coord - zoom_size/2))
    y_end = int(min(img.shape[0], y_coord + zoom_size/2))
    
    zoomed = img[y_start:y_end, x_start:x_end]
    ax2.imshow(zoomed, cmap='gray')
    
    # Add crosshair to zoomed region
    center_x = x_coord - x_start
    center_y = y_coord - y_start
    ax2.axvline(x=center_x, color='red', alpha=0.5)
    ax2.axhline(y=center_y, color='red', alpha=0.5)
    ax2.plot(center_x, center_y, 'r+', markersize=10, markeredgewidth=2)
    
    ax1.axis('off')
    ax2.axis('off')
    ax1.set_title('Full Image')
    ax2.set_title('Zoomed Region')
    
    plt.tight_layout()
    plt.show()

## Checking the imbalanceness of the data

In [22]:
target_variables = ['left_infraclinoid_internal_carotid_artery',  'right_infraclinoid_internal_carotid_artery'
                    ,  'left_supraclinoid_internal_carotid_artery',  'right_supraclinoid_internal_carotid_artery'
                    ,  'left_middle_cerebral_artery',  'right_middle_cerebral_artery'
                    ,  'anterior_communicating_artery',  'left_anterior_cerebral_artery'
                    ,  'right_anterior_cerebral_artery',  'left_posterior_communicating_artery'
                    ,  'right_posterior_communicating_artery'
                    ,  'basilar_tip',  'other_posterior_circulation'
                   ]

In [23]:
df_agg = df_all_data.group_by(target_variables).agg(pl.col("file_name").count())

bias_arr_prep_dict = {}
for pos in target_variables:
    print(pos)
    #print(df_agg.filter(pl.col(pos)==1).select(pl.col('file_name')).sum().item(0,0))
    bias_arr_prep_dict[pos] = df_agg.filter(pl.col(pos)==1).select(pl.col('file_name')).sum().item(0,0)
    
print(bias_arr_prep_dict)

left_infraclinoid_internal_carotid_artery
right_infraclinoid_internal_carotid_artery
left_supraclinoid_internal_carotid_artery
right_supraclinoid_internal_carotid_artery
left_middle_cerebral_artery
right_middle_cerebral_artery
anterior_communicating_artery
left_anterior_cerebral_artery
right_anterior_cerebral_artery
left_posterior_communicating_artery
right_posterior_communicating_artery
basilar_tip
other_posterior_circulation
{'left_infraclinoid_internal_carotid_artery': 117, 'right_infraclinoid_internal_carotid_artery': 154, 'left_supraclinoid_internal_carotid_artery': 456, 'right_supraclinoid_internal_carotid_artery': 374, 'left_middle_cerebral_artery': 320, 'right_middle_cerebral_artery': 429, 'anterior_communicating_artery': 446, 'left_anterior_cerebral_artery': 62, 'right_anterior_cerebral_artery': 80, 'left_posterior_communicating_artery': 133, 'right_posterior_communicating_artery': 146, 'basilar_tip': 163, 'other_posterior_circulation': 166}


In [24]:
total_pos = 0
for pos in target_variables:
    total_pos = total_pos + bias_arr_prep_dict[pos]

print(total_pos)

3046


In [25]:
bias_dict = {}
bias_list = []
for pos in target_variables:
    local_pos = total_pos - bias_arr_prep_dict[pos]
    bias_dict[pos] = round(np.log(bias_arr_prep_dict[pos])/local_pos,5)
    bias_list.append(bias_dict[pos])

print(bias_dict)
print(bias_list)

{'left_infraclinoid_internal_carotid_artery': 0.00163, 'right_infraclinoid_internal_carotid_artery': 0.00174, 'left_supraclinoid_internal_carotid_artery': 0.00236, 'right_supraclinoid_internal_carotid_artery': 0.00222, 'left_middle_cerebral_artery': 0.00212, 'right_middle_cerebral_artery': 0.00232, 'anterior_communicating_artery': 0.00235, 'left_anterior_cerebral_artery': 0.00138, 'right_anterior_cerebral_artery': 0.00148, 'left_posterior_communicating_artery': 0.00168, 'right_posterior_communicating_artery': 0.00172, 'basilar_tip': 0.00177, 'other_posterior_circulation': 0.00177}
[0.00163, 0.00174, 0.00236, 0.00222, 0.00212, 0.00232, 0.00235, 0.00138, 0.00148, 0.00168, 0.00172, 0.00177, 0.00177]


In [27]:
def get_bias_val():
    bias_moderate = round(np.log(bias_arr_prep_dict['Moderate']/(bias_arr_prep_dict['Severe'] + bias_arr_prep_dict['Normal/Mild'])),5)
    bias_norm_mild = round(np.log(bias_arr_prep_dict['Normal/Mild']/(bias_arr_prep_dict['Severe'] + bias_arr_prep_dict['Moderate'])),5)
    bias_severe = round(np.log(bias_arr_prep_dict['Severe']/(bias_arr_prep_dict['Moderate'] + bias_arr_prep_dict['Normal/Mild'])),5)
    
    return [bias_moderate, bias_norm_mild, bias_severe]

bias_arr = get_bias_val()
print(bias_arr)

163

In [17]:

df_all_data.select(target_variables).head(5)

left_infraclinoid_internal_carotid_artery,right_infraclinoid_internal_carotid_artery,left_supraclinoid_internal_carotid_artery,right_supraclinoid_internal_carotid_artery,left_middle_cerebral_artery,right_middle_cerebral_artery,anterior_communicating_artery,left_anterior_cerebral_artery,right_anterior_cerebral_artery,left_posterior_communicating_artery,right_posterior_communicating_artery,basilar_tip,other_posterior_circulation
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0,0,0,0,0,1,0,1,0,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
dd.sql(" \
select t1.modality, t1.aneurysm_present_in_image, t1.per_mod_count, \
round(t1.per_mod_count/t2.total_count ,3) as modality_pct \
from \
( \
select modality, aneurysm_present_in_image, cast(count(1) as float) as per_mod_count from df_all_data \
group by modality, aneurysm_present_in_image \
)t1 \
join \
(select cast(count(1) as float) as total_count from df_all_data)t2 \
on 1=1 \
order by 1 \
").pl()

modality,aneurysm_present_in_image,per_mod_count,modality_pct
str,i32,f32,f32
"""CTA""",1,1221.0,0.001
"""CTA""",0,725991.0,0.725
"""MRA""",0,195928.0,0.196
"""MRA""",1,657.0,0.001
"""MRI T1post""",0,47105.0,0.047
"""MRI T1post""",1,92.0,0.0
"""MRI T2""",1,279.0,0.0
"""MRI T2""",0,30073.0,0.03


In [13]:
def read_and_parse_dicom_files_tensorflow_train(dcm_path, f_coord=None):
    raw_image = tf.io.read_file(dcm_path)
    image = tfio.image.decode_dicom_image(raw_image, scale='auto', dtype=tf.float32)
    #if f_coord:
    specific_slice_tensor = tf.slice(
        image,
        begin=[f_coord, 0, 0, 0],
        size=[1, image.shape[1], image.shape[2], image.shape[3]]
    )
    # Squeeze to remove the extra dimension of size 1
    specific_slice_tensor = tf.squeeze(specific_slice_tensor)
    """else:
        specific_slice_tensor = tf.squeeze(image)"""
    expanded_image = tf.expand_dims(specific_slice_tensor, -1)
    m, M=tf.math.reduce_min(expanded_image), tf.math.reduce_max(expanded_image)
    expanded_image = (tf.image.grayscale_to_rgb(expanded_image)-m)/(M-m)
    expanded_image = tf.image.resize(expanded_image, (128,128))
    sqzd_image = tf.squeeze(expanded_image)
    return sqzd_image

def load_dicom_tf(filepath_tensor, f_coord_tensor=None):
    """
    Wrapper function to call read_and_parse_dicom_files_tensorflow_train within a tf.data.Dataset pipeline.
    """
    #if f_coord_tensor:
    image = tf.py_function(
        read_and_parse_dicom_files_tensorflow_train,
        inp=[filepath_tensor, f_coord_tensor],
        Tout=tf.float32 # Adjust Tout based on the expected output type of your image data
    )
    """else:
        image = tf.py_function(
            read_and_parse_dicom_files_tensorflow_train,
            inp=[filepath_tensor],
            Tout=tf.float32 # Adjust Tout based on the expected output type of your image data
    )"""
    # Ensure the shape is defined if known, as tf.py_function can lose shape info
    #image.set_shape([128, 128]) # Example for a 2D image, adjust as needed
    return image 

def preprocessing(dcm_path, f_coord):
    train_img = load_dicom_tf(dcm_path, f_coord)
    #train_img = read_and_parse_dicom_files_tensorflow_train(dcm_path, f_coord)
    train_img = tf.reshape(train_img, shape=(128, 128, 3))
    return train_img

def load_dataset_tensorflow_train(dcm_path, f_coord, labels):
    image = preprocessing(dcm_path, f_coord)
    return {"images": tf.cast(image, tf.float32), "labels": tf.cast(labels, tf.float32)}

def dict_to_tuple(inputs):
    return inputs["images"], inputs["labels"]

In [14]:
cols_to_select = ['full_image_path', 'coordinates_f', 'left_infraclinoid_internal_carotid_artery',  'right_infraclinoid_internal_carotid_artery',  'left_supraclinoid_internal_carotid_artery'
                  ,  'right_supraclinoid_internal_carotid_artery',  'left_middle_cerebral_artery',  'right_middle_cerebral_artery',  'anterior_communicating_artery'
                  ,  'left_anterior_cerebral_artery',  'right_anterior_cerebral_artery',  'left_posterior_communicating_artery',  'right_posterior_communicating_artery'
                  ,  'basilar_tip',  'other_posterior_circulation'
]

df_for_train_baseline_cta = df_all_data.filter(pl.col('modality')=='CTA').select(cols_to_select)
df_for_train_baseline_mri_t2 = df_all_data.filter(pl.col('modality')=='MRI T2').select(cols_to_select)
df_for_train_baseline_mra = df_all_data.filter(pl.col('modality')=='MRA').select(cols_to_select)
df_for_train_baseline_mri_t1post = df_all_data.filter(pl.col('modality')=='MRI T1post').select(cols_to_select)

In [15]:
x_train_cta, x_test_val_cta = train_test_split(df_for_train_baseline_cta, test_size=0.4, random_state=42)
x_test_cta, x_valid_cta = train_test_split(x_test_val_cta, test_size=0.2, random_state=42)

x_train_mri_t2, x_test_val_mri_t2 = train_test_split(df_for_train_baseline_mri_t2, test_size=0.4, random_state=42)
x_test_mri_t2, x_valid_mri_t2 = train_test_split(x_test_val_mri_t2, test_size=0.2, random_state=42)

x_train_mra, x_test_val_mra = train_test_split(df_for_train_baseline_mra, test_size=0.4, random_state=42)
x_test_mra, x_valid_mra = train_test_split(x_test_val_mra, test_size=0.2, random_state=42)

x_train_mri_t1post, x_test_val_mri_t1post = train_test_split(df_for_train_baseline_mri_t1post, test_size=0.4, random_state=42)
x_test_mri_t1post, x_valid_mri_t1post = train_test_split(x_test_val_mri_t1post, test_size=0.2, random_state=42)

print("********* CTA ****************")
print("Training data shape : {0}".format(x_train_cta.shape))
print("Test data shape : {0}".format(x_test_cta.shape))
print("Validation data shape : {0}".format(x_valid_cta.shape))

print("********* MRI T2 ****************")
print("Training data shape : {0}".format(x_train_mri_t2.shape))
print("Test data shape : {0}".format(x_test_mri_t2.shape))
print("Validation data shape : {0}".format(x_valid_mri_t2.shape))

print("********* MRA ****************")
print("Training data shape : {0}".format(x_train_mra.shape))
print("Test data shape : {0}".format(x_test_mra.shape))
print("Validation data shape : {0}".format(x_valid_mra.shape))

print("********* MRI T1post ****************")
print("Training data shape : {0}".format(x_train_mri_t1post.shape))
print("Test data shape : {0}".format(x_test_mri_t1post.shape))
print("Validation data shape : {0}".format(x_valid_mri_t1post.shape))

********* CTA ****************
Training data shape : (436327, 15)
Test data shape : (232708, 15)
Validation data shape : (58177, 15)
********* MRI T2 ****************
Training data shape : (18211, 15)
Test data shape : (9712, 15)
Validation data shape : (2429, 15)
********* MRA ****************
Training data shape : (117951, 15)
Test data shape : (62907, 15)
Validation data shape : (15727, 15)
********* MRI T1post ****************
Training data shape : (28318, 15)
Test data shape : (15103, 15)
Validation data shape : (3776, 15)


In [16]:
label_cols = ['left_infraclinoid_internal_carotid_artery',  'right_infraclinoid_internal_carotid_artery',  'left_supraclinoid_internal_carotid_artery'
                  ,  'right_supraclinoid_internal_carotid_artery',  'left_middle_cerebral_artery',  'right_middle_cerebral_artery',  'anterior_communicating_artery'
                  ,  'left_anterior_cerebral_artery',  'right_anterior_cerebral_artery',  'left_posterior_communicating_artery',  'right_posterior_communicating_artery'
                  ,  'basilar_tip',  'other_posterior_circulation'
]

In [17]:
def generate_tf_datasets(p_df, p_BATCH_SIZE_PER_REPLICA):

    BATCH_SIZE = p_BATCH_SIZE_PER_REPLICA * tpu_strategy.num_replicas_in_sync
    
    image_filenames = pl.Series(p_df.select(pl.col('full_image_path'))).to_list()
    image_f_coords = pl.Series(p_df.select(pl.col('coordinates_f'))).fill_null(0).to_list()
    image_labels = pl.Series(p_df.select(label_cols)).map_elements(lambda d: list(d.values())).to_list()
    
    image_dataset = tf.data.Dataset.from_tensor_slices((image_filenames, image_f_coords, image_labels))
    
    image_ds = image_dataset.map(load_dataset_tensorflow_train, num_parallel_calls=tf.data.AUTOTUNE)
    image_ds = image_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    image_ds = image_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
    image_ds = image_ds.prefetch(tf.data.AUTOTUNE)
    
    return image_ds

In [34]:
x_train_trial = x_train.slice(offset=0, length=1000)
x_test_trial = x_test.slice(offset=0, length=1000)
x_valid_trial = x_valid.slice(offset=0, length=1000)
x_train_trial.shape

(1000, 15)

In [18]:
train_ds = generate_tf_datasets(p_df=x_train_mri_t1post, p_BATCH_SIZE_PER_REPLICA = 16)
test_ds = generate_tf_datasets(p_df=x_test_mri_t1post, p_BATCH_SIZE_PER_REPLICA = 16)
valid_ds = generate_tf_datasets(p_df=x_valid_mri_t1post, p_BATCH_SIZE_PER_REPLICA = 16)

In [31]:
#from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers

with tpu_strategy.scope():
    
    rsna_input = layers.Input(shape=(128,128,3), name="rsna_input")

    #conv_base = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=rsna_input)
    conv_base = InceptionV3(include_top=False, weights="imagenet", input_tensor=rsna_input, input_shape=(128,128,3))
    conv_base.trainable = False

    #max_pool_layer_0 = layers.MaxPooling2D(name="max_pool_0", pool_size=(2, 2), strides=(1, 1), padding="same")(conv_base.output)
    #max_pool_layer_0 = layers.BatchNormalization()(conv_base.output)

    #conv2d_1 = layers.Conv2D(filters=100, kernel_size=4, strides=1, padding="same", activation="relu")(max_pool_layer_0)
    #max_pool_layer_1 = layers.MaxPooling2D(name="max_pool_1", pool_size=(2, 2), strides=(1, 1), padding="same")(conv2d_1)
    #max_pool_layer_1 = layers.BatchNormalization()(max_pool_layer_1)

    flattened_layer = layers.Flatten()(conv_base.output)

    #hidden_layer1 = layers.Dense(200, activation="selu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(flattened_layer)
    #hidden_layer1 = layers.BatchNormalization()(hidden_layer1)
    #hidden_layer2 = layers.LeakyReLU(alpha=0.01)(hidden_layer1) 
    #hidden_layer2 = layers.Dense(100, activation="selu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(hidden_layer1)
    #hidden_layer2 = layers.BatchNormalization()(hidden_layer2)
    #hidden_layer3 = layers.Dense(50, activation="selu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(hidden_layer2)
    #hidden_layer3 = layers.BatchNormalization()(hidden_layer3)
    rsna_output = layers.Dense(13, activation="softmax")(flattened_layer)
    model1 = tf.keras.Model(rsna_input, rsna_output)

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("keras_effnet_mri_t1post.keras")
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

    model1.compile(loss="binary_crossentropy", optimizer="adamax", metrics=["accuracy"])

In [33]:
model1.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ rsna_input          │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_471 (Conv2D) │ (None, 63, 63,    │        864 │ rsna_input[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 63, 63,    │         96 │ conv2d_471[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_470      │ (None, 63, 63,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_472 (Conv2D) │ (None, 61, 61,    │      9,216 │ activation_470[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 61, 61,    │         96 │ conv2d_472[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_471      │ (None, 61, 61,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_473 (Conv2D) │ (None, 61, 61,    │     18,432 │ activation_471[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 61, 61,    │        192 │ conv2d_473[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_472      │ (None, 61, 61,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_20    │ (None, 30, 30,    │          0 │ activation_472[0… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_474 (Conv2D) │ (None, 30, 30,    │      5,120 │ max_pooling2d_20… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 30, 30,    │        240 │ conv2d_474[0][0]  │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_473      │ (None, 30, 30,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_475 (Conv2D) │ (None, 28, 28,    │    138,240 │ activation_473[0… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 28,    │        576 │ conv2d_475[0][0]  │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_474      │ (None, 28, 28,    │          0 │ batch_normalizat

 Total params: 22,122,313 (84.39 MB)

 Trainable params: 106,509 (416.05 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

 Optimizer params: 213,020 (832.11 KB)

In [32]:
with tpu_strategy.scope():
    history = model1.fit(train_ds, epochs=15, \
                         validation_data=valid_ds, \
                         callbacks=[checkpoint_cb, early_stopping_cb])

W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers t

Epoch 1/15


I0000 00:00:1758777639.117937    1419 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(717fa0e2c9b1afd3:0:0), session_name()
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel 

  1/221 ━━━━━━━━━━━━━━━━━━━━ 48:08 13s/step - accuracy: 0.1875 - loss: 1.1524

W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result


...

W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result


 64/221 ━━━━━━━━━━━━━━━━━━━━ 58s 373ms/step - accuracy: 0.5125 - loss: nan

W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers t

KeyboardInterrupt: 

W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (

In [21]:
with tpu_strategy.scope():
    model1.evaluate(test_ds)

W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (

  1/117 ━━━━━━━━━━━━━━━━━━━━ 1:34 813ms/step - accuracy: 1.0000 - loss: nan

W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create une

  2/117 ━━━━━━━━━━━━━━━━━━━━ 1:23 730ms/step - accuracy: 1.0000 - loss: nan

W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers t

...

115/117 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step - accuracy: 0.9997 - loss: nan

W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers t

116/117 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step - accuracy: 0.9997 - loss: nan

W: processing MR image ... applying modality transform may create unexpected result
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: processing MR image ... applying modality transform may create unexpected result
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value
W: invalid value for 'OverlayBitPosition' (12), refers to bit position within stored pixel value


117/117 ━━━━━━━━━━━━━━━━━━━━ 51s 432ms/step - accuracy: 0.9997 - loss: nan


In [46]:
def read_and_parse_dicom_files_tensorflow_infer(dcm_path, f_coord=None):
    #tf.config.run_functions_eagerly(True)
    raw_image = tf.io.read_file(dcm_path)
    image = tfio.image.decode_dicom_image(raw_image, scale='auto', dtype=tf.float32)
    #if f_coord:
    specific_slice_tensor = tf.slice(
        image,
        begin=[f_coord, 0, 0, 0],
        size=[1, image.shape[1], image.shape[2], image.shape[3]]
    )
    # Squeeze to remove the extra dimension of size 1
    specific_slice_tensor = tf.squeeze(specific_slice_tensor)
    """else:
        specific_slice_tensor = tf.squeeze(image)"""
    #print("specific_slice_tensor - ", specific_slice_tensor.shape)
    expanded_image = tf.expand_dims(specific_slice_tensor, -1)
    m, M=tf.math.reduce_min(expanded_image), tf.math.reduce_max(expanded_image)
    expanded_image = (tf.image.grayscale_to_rgb(expanded_image)-m)/(M-m)
    expanded_image = tf.image.resize(expanded_image, (128,128))
    sqzd_image = tf.squeeze(expanded_image)
    #print("sqzd_image -", sqzd_image.shape)
    return sqzd_image

In [51]:
image_filenames = pl.Series(x_valid.select(pl.col('full_image_path'))).to_list()[0]
image_f_coords = pl.Series(x_valid.select(pl.col('coordinates_f'))).fill_null(0).to_list()[0]
image_labels = pl.Series(x_valid.select(label_cols)).map_elements(lambda d: list(d.values())).to_list()[0]

In [71]:
image_filenames

'/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.87793717347274644321991233773637029287/1.2.826.0.1.3680043.8.498.99151007284580747641718722709119849751.dcm'

In [57]:
test_image = load_dicom_tf(image_filenames, image_f_coords)
#read_and_parse_dicom_files_tensorflow_infer(image_filenames, image_f_coords)

In [58]:
test_image.shape

TensorShape([128, 128, 3])

In [59]:
test_image_for_inf = np.expand_dims(test_image, axis=0)
test_image_for_inf.shape

(1, 128, 128, 3)

In [66]:
#tf.config.run_functions_eagerly(False)
prediction = model1.predict(test_image_for_inf)

2025-09-14 13:26:38.893122: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node functional_1_1/batch_normalization_100_1/Cast/ReadVariableOp.
I0000 00:00:1757856399.484750    1101 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(65ff671008c06d5f:0:0), session_name()
I0000 00:00:1757856404.089515    1101 tpu_compile_op_common.cc:245] Compilation of 65ff671008c06d5f:0:0 with session name  took 4.604704404s and succeeded
I0000 00:00:1757856404.105053    1101 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(65ff671008c06d5f:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_1932975749922622089", property.function_library_fingerprint = 7020109480359404312, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chi

1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step


In [67]:
prediction.shape

(1, 13)

In [70]:
prediction[0]

array([0.01617235, 0.7627834 , 0.0816794 , 0.08579633, 0.60275835,
       0.01867604, 0.10895562, 0.21828824, 0.18612197, 0.7638618 ,
       0.49959147, 0.9609399 , 0.3264107 ], dtype=float32)

In [69]:
image_labels

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]